In [ ]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [2]:
from torchmetrics.functional.classification import multilabel_f1_score
from torch import tensor
import torch
target = tensor([[0,1,1,0,0,0,0,0,0,0]])
preds = tensor([[0.3755, 0.7549, 0.8196, 0.1965, 0.3098, 0.1127, 0.4669, 0.2901, 0.38519,
         0.4704]])

In [4]:
target = torch.randint(0, 2, (10, 100))

In [13]:
target.sum(1)

tensor([56, 44, 57, 48, 54, 38, 52, 51, 45, 51])

In [35]:
multilabel_f1_score(preds, target, num_labels=10, average='macro', threshold=0.5)

tensor(0.2000)

In [4]:
target.shape, preds.shape

(torch.Size([8, 6, 1]), torch.Size([8, 6, 1]))

In [42]:
torch.sigmoid(tensor([100]))

tensor([1.])

: 

In [5]:
from argparse import ArgumentParser
import pandas as pd
from urllib.request import urlopen
from PIL import Image
import timm
import torch
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import csv

In [6]:
class_mapping = "/home/ubuntu/FGVC11/data/PlantClef/pretrained_models/class_mapping.txt"
species_mapping = "/home/ubuntu/FGVC11/data/PlantClef/pretrained_models/species_id_to_name.txt"
pretrained_path = "/home/ubuntu/FGVC11/data/PlantClef/pretrained_models/vit_base_patch14_reg4_dinov2_lvd142m_pc24_onlyclassifier_then_all/model_best.pth.tar"

In [7]:
submission_df = pd.DataFrame(columns=["path", "plot_id", "species_ids"])
submission_df["path"] = glob("/home/ubuntu/FGVC11/data/PlantClef/images/*.jpg")
submission_df["plot_id"] = submission_df["path"].apply(lambda x: x.split("/")[-1].split(".")[0])

In [8]:
def load_class_mapping(class_list_file):
    with open(class_list_file) as f:
        class_index_to_class_name = {i: line.strip() for i, line in enumerate(f)}
    return class_index_to_class_name


def load_species_mapping(species_map_file):
    df = pd.read_csv(species_map_file, sep=';', quoting=1, dtype={'species_id': str})
    df = df.set_index('species_id')
    return  df['species'].to_dict()

In [9]:
cid_to_spid = load_class_mapping(class_mapping)
spid_to_sp = load_species_mapping(species_mapping)
    
device = "cuda"

model = timm.create_model('vit_base_patch14_reg4_dinov2.lvd142m', pretrained=False, 
                          num_classes=len(cid_to_spid), checkpoint_path=pretrained_path)
model = model.to(device)
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

In [10]:
transforms

Compose(
    Resize(size=518, interpolation=bicubic, max_size=None, antialias=warn)
    CenterCrop(size=(518, 518))
    ToTensor()
    Normalize(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]))
)

In [11]:
??model.forward_features

Signature: model.forward_features(x: torch.Tensor) -> torch.Tensor
Docstring: <no docstring>
Source:   
    def forward_features(self, x: torch.Tensor) -> torch.Tensor:
        x = self.patch_embed(x)
        x = self._pos_embed(x)
        x = self.patch_drop(x)
        x = self.norm_pre(x)
        if self.grad_checkpointing and not torch.jit.is_scripting():
            x = checkpoint_seq(self.blocks, x)
        else:
            x = self.blocks(x)
        x = self.norm(x)
        return x
File:      ~/miniconda3/envs/transformers/lib/python3.10/site-packages/timm/models/vision_transformer.py
Type:      method

In [12]:
??model.forward_head

Signature: model.forward_head(x: torch.Tensor, pre_logits: bool = False) -> torch.Tensor
Docstring: <no docstring>
Source:   
    def forward_head(self, x: torch.Tensor, pre_logits: bool = False) -> torch.Tensor:
        if self.attn_pool is not None:
            x = self.attn_pool(x)
        elif self.global_pool == 'avg':
            x = x[:, self.num_prefix_tokens:].mean(dim=1)
        elif self.global_pool:
            x = x[:, 0]  # class token
        x = self.fc_norm(x)
        x = self.head_drop(x)
        return x if pre_logits else self.head(x)
File:      ~/miniconda3/envs/transformers/lib/python3.10/site-packages/timm/models/vision_transformer.py
Type:      method

In [13]:
??model.reset_classifier

Signature: model.reset_classifier(num_classes: int, global_pool=None) -> None
Docstring: <no docstring>
Source:   
    def reset_classifier(self, num_classes: int, global_pool = None) -> None:
        self.num_classes = num_classes
        if global_pool is not None:
            assert global_pool in ('', 'avg', 'token', 'map')
            if global_pool == 'map' and self.attn_pool is None:
                assert False, "Cannot currently add attention pooling in reset_classifier()."
            elif global_pool != 'map ' and self.attn_pool is not None:
                self.attn_pool = None  # remove attention pooling
            self.global_pool = global_pool
        self.head = nn.Linear(self.embed_dim, num_classes) if num_classes > 0 else nn.Identity()
File:      ~/miniconda3/envs/transformers/lib/python3.10/site-packages/timm/models/vision_transformer.py
Type:      method

In [ ]:
from torch.utils.data import Dataset, DataLoader
class PlantClefDataset(Dataset):
    def __init__(self, df, transforms):
        self.df = df
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row["path"]).convert("RGB")
        img = self.transforms(img)
        return img

# Create the dataset
dataset = PlantClefDataset(submission_df, transforms)

# Create the dataloader
batch_size = 8
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=8)


In [ ]:
def forward_head(self, x, pool=None) -> torch.Tensor:
    # x = x[:, self.num_prefix_tokens:]
    # convert the second dimension two b, 37 * 37, 768 then apply max pooling to get b, 9 x 9, 768 and then resize back to b, 81, 768
    # print(x.shape)
    # x = x.view(x.size(0), x.size(-1), 37, 37)
    if pool:
        x = x.permute(0,2,1)
        # print(x.shape)
        x = torch.nn.functional.avg_pool1d(x, kernel_size=pool, stride=pool)
        # print(x.shape)
        # x = x.view(x.size(0), -1, x.size(1)) 
        x = x.permute(0,2,1)
    # print(x.shape) 
    x = self.fc_norm(x)
    # print(x.shape)
    x = self.head_drop(x)
    # print(x.shape)
    return self.head(x)

model.forward_head = forward_head

In [ ]:
test_labels = []
for data in tqdm(dataloader):
    with torch.no_grad():
        feats = model.forward_features(data.to(device))[:, model.num_prefix_tokens:] #.detach().cpu()
        # feats_lr = torch.pca_lowrank(feats, q=30)
        output = model.forward_head(model, feats, pool=8).detach().cpu()
        max_prob = torch.argmax(output, dim=2).numpy()
        # idxs = torch.unique(torch.argmax(output[0], dim=1))
        # output = torch.sigmoid(model(data.to(device))).detach().cpu()
        # _, idxs = torch.topk(output, k=8, dim=1)
        for idx in max_prob:
            unique_idx, count = np.unique(idx, return_counts=True)
            unique_idx = unique_idx[np.argsort(-count)]
            test_labels.append(str([int(cid_to_spid[j]) for k,j in enumerate(unique_idx) if k < 7]))
            # test_labels.append(str([int(cid_to_spid[j]) for j in torch.unique(idx).numpy()]))
    # break

In [ ]:
test_labels

In [ ]:
spids = [str(cid_to_spid[j]) for k,j in enumerate(unique_idx) if k < 12]
[spid_to_sp[i] for i in spids]

In [ ]:
test_labels[0]

In [ ]:
submission_df["species_ids"] = test_labels
submission_df[["plot_id", "species_ids"]].to_csv("my_run.csv", sep=';', index=False, quoting=csv.QUOTE_NONE)

In [ ]:
submission_df

### Checking with training data

In [14]:
df_train = pd.read_csv("/home/ubuntu/FGVC11/data/PlantClef/PlantCLEFTrainLQ.csv",  delimiter=";", escapechar="/")

/tmp/ipykernel_178783/434157496.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv("/home/ubuntu/FGVC11/data/PlantClef/PlantCLEFTrainLQ.csv",  delimiter=";", escapechar="/")


In [ ]:
model.global_pool = "avg"

In [16]:
def forward_head(self, x, pool=None) -> torch.Tensor:
    # x = x[:, self.num_prefix_tokens:]
    # convert the second dimension two b, 37 * 37, 768 then apply max pooling to get b, 9 x 9, 768 and then resize back to b, 81, 768
    # print(x.shape)
    # x = x.view(x.size(0), x.size(-1), 37, 37)
    if pool:
        x = x.permute(0,2,1)
        # print(x.shape)
        x = torch.nn.functional.avg_pool1d(x, kernel_size=pool, stride=pool)
        # print(x.shape)
        # x = x.view(x.size(0), -1, x.size(1)) 
        x = x.permute(0,2,1)
    # print(x.shape) 
    x = self.fc_norm(x)
    # print(x.shape)
    x = self.head_drop(x)
    # print(x.shape)
    return self.head(x)

model.forward_head = forward_head

In [15]:
# tgt = []
# pred = []
# for i, row in tqdm(df_train.iterrows(), total=len(df_train)):
#     img = Image.open(row["path"]).convert("RGB")
#     img = transforms(img).unsqueeze(0).to(device)
#     with torch.no_grad():
#         feats = model.forward_features(img)
#         out = model.forward_head(feats)
#         output = torch.sigmoid(out).detach().cpu().numpy()[0]
#         max_prob_idx = np.argmax(output)
#         pred.append(int(cid_to_spid[max_prob_idx]))
#         tgt.append(row["species_id"])
#     if i == 100:
#         break
import random
model.eval();
tgt = []
pred = []
for i, row in tqdm(df_train.iterrows(), total=len(df_train)):
    row_ = df_train.iloc[random.randint(0, len(df_train)-1)]
    img = Image.open(row_["path"]).convert("RGB")
    img = transforms(img).unsqueeze(0).to(device)
    with torch.no_grad():
        feats = model.forward_features(img.to(device))[:, model.num_prefix_tokens:] #.detach().cpu()
        output = model.forward_head(model, feats, pool=16).detach().cpu()
        max_prob = torch.argmax(output, dim=2)
        # choose the one with most frequency
        max_prob_idx = torch.mode(max_prob, dim=1).values.numpy()[0]
        pred.append(int(cid_to_spid[max_prob_idx]))
        tgt.append(row_["species_id"])
    if i == 100:
        break

  0%|          | 0/1408033 [00:01<?, ?it/s]


TypeError: VisionTransformer.forward_head() got an unexpected keyword argument 'pool'

In [ ]:
# pred, tgt

In [ ]:
pred = np.array(pred)
tgt = np.array(tgt)

In [ ]:
pred.shape, tgt.shape

In [ ]:
sum(pred == tgt)/len(pred)

In [ ]:
sum(pred == tgt)/len(pred)

In [ ]:
sum(pred == tgt)/len(pred)

In [ ]:
transforms

In [ ]:
model.patch_embed

In [ ]:
import torchsummary
torchsummary.summary(model, (3, 518, 518))